In [3]:
import numpy as np
from numpy import polyval, place, extract, any, asarray, nan, inf, pi
from numpy import (where, arange, putmask, ravel, sum, shape,
                   log, sqrt, exp, arctanh, tan, sin, arcsin, arctan,
                   tanh, cos, cosh, sinh, log1p, expm1)
import pystan
from pystan import StanModel 
from scipy.stats import rv_continuous

class LogGamma_gen(rv_continuous):
#     def _argcheck(self, c):
#         c = asarray(c)
#         self.b = where(c < 0, 1.0/abs(c), inf)
#         return where(c == 0, 0, 1)

    def _pdf(self, x):
#         ax = abs(x)
        Px = pow(x, -2) * pow(log(x),-1)
        return Px

#     def _logpdf(self, x, alpha1, alpha2, beta):
#         return (alpha1 - 1) * np.log(x) - alpha1 * np.log(beta) - np.log(ss.beta(alpha1, alpha2)) - (alpha1 + alpha2) * np.log(1 + x/beta)

#     def _cdf(self, x, alpha):
#         return 1 - pow( alpha / ( alpha + pow(x, alpha) ), alpha)

#     def _ppf(self, q, c):
#         vals = 1.0/c * (pow(1-q, -c)-1)
#         return vals

#     def _munp(self, n, c):
#         k = arange(0, n+1)
#         val = (-1.0/c)**n * sum(comb(n, k)*(-1)**k / (1.0-c*k), axis=0)
#         return where(c*n < 1, val, inf)

#     def _entropy(self, c):
#         if (c > 0):
#             return 1+c
#         else:
#             self.b = -1.0 / c
#             return rv_continuous._entropy(self, c)
LogGamma = LogGamma_gen(a=0.1, name='LogGamma') # (a=0.1, b=... ,  name='LogGamma') here we specify the support of pdf [a,b]

In [4]:
N, k, alpha = 100, 100, 2
r = LogGamma.rvs(size = N)
r

C:\Users\Michal Lewandowski\Anaconda3\envs\py35\lib\site-packages\scipy\integrate\quadpack.py:364: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  warnings.warn(msg, IntegrationWarning)
C:\Users\Michal Lewandowski\Anaconda3\envs\py35\lib\site-packages\scipy\integrate\quadpack.py:364: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  warnings.warn(msg, IntegrationWarning)
C:\Users\Michal Lewandowski\Anaconda3\envs\py35\lib\site-packages\scipy\integrate\quadpack.py:364: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singular

array([8109.00006221, 9773.61609315, 7822.23286271, 8662.51066117,
       6519.69483006, 8414.52181102, 6631.81709263, 6986.26427934,
       8041.8201064 , 6167.13233795, 7239.4011119 , 6745.68701023,
       9946.27766661, 7846.7732721 , 8380.50824355, 9662.03647242,
       9594.79542797, 8956.75311903, 7953.53218808, 6788.89988103,
       7352.40396236, 6256.89753148, 9963.59167066, 9738.14953309,
       4571.4101105 , 9830.5       , 9830.5       , 7107.75721269,
       4518.1390801 , 8780.32864603, 9909.46513438, 9290.22514744,
       8200.91560345, 6298.34967452, 8536.11646077, 8268.05819475,
       8134.34680121, 9063.30810176, 9712.73568786, 9067.04168782,
       7100.8660262 , 8876.25838257, 7372.9       , 9999.99642975,
       9614.28243177, 9413.41248215, 6885.25493753, 8623.63008812,
       4690.28121705, 7409.69577893, 9992.33206386, 9614.28243177,
       6587.76040274, 7736.97982639, 9773.61609315, 6855.71269321,
       8956.75311903, 9999.76516556, 8859.85308126, 9997.79377

In [5]:
from prettytable import PrettyTable
m = 3 # how much we round
t = PrettyTable(['pr.distr.', 'mean', 'sd', '2.5%', '25%', '50%', '75%', '97.5%'])
t.add_row(['Burr',
           "%.3f" % round(np.mean(r),m),
           "%.3f" % round(np.std(r),m),
           "%.3f" % round(np.percentile(r, 2.5),m), 
           "%.3f" % round(np.percentile(r, 25),m), 
           "%.3f" % round(np.percentile(r, 50),m), 
           "%.3f" % round(np.percentile(r, 75),m), 
           "%.3f" % round(np.percentile(r, 97.5),m)])
# t.add_row(['Bob', 19])
print(t)

+-----------+----------+----------+----------+----------+----------+----------+----------+
| pr.distr. |   mean   |    sd    |   2.5%   |   25%    |   50%    |   75%    |  97.5%   |
+-----------+----------+----------+----------+----------+----------+----------+----------+
|    Burr   | 8185.388 | 1590.646 | 4543.443 | 7106.034 | 8475.319 | 9635.445 | 9997.794 |
+-----------+----------+----------+----------+----------+----------+----------+----------+


In [ ]:
def k_greatest_values(a,k):
    """returns k greatest elements from the list and k-1 value starting from which we consider values to be extreme"""
    u = np.sort(a, axis=None)[-1-k]
    a = np.sort(a, axis=None)[-1-k+1:]
    a = asarray([a-u for x in a])
    return(a[1].tolist(), u) # u the starting value from which we consider others as excesses

In [ ]:
k = 100 # number of excesses
log_gamma, u = k_greatest_values(r, k)
# frechet # so we recover  k = 100 excesses sampled from Frechet distribution

In [ ]:
import pystan
from pystan import StanModel 
GPD = """
functions {
  real myGPD_lpdf(real y, real alpha, real beta) {
      //return -(1+1/c)*log(1+c*y2);
      return -(alpha + 1)*( log(1+y/beta) )+(log(alpha) - log(beta));
  }
  // above distribution is a special case of the distribution in the paper for alpha = beta = 1 / c
  // c = 1/beta
}
data { 
  int N;
  real y[N]; // points sampled from gpd in python with some(known) parameters, by mcmc we recover true values of those params
}
parameters { 
  //real c; 
  real alpha;
  real beta;
}
model {
  // Priors
  // no priors - we assume improper priors on params
  // c ~ uniform(0,1);

// Likelihood
  for(n in 1:N) {
    target += myGPD_lpdf( y[n] | alpha, beta );
  }

}
generated quantities{}
"""

In [ ]:
data = dict(N = k,  y = log_gamma) 
fit = StanModel(model_code=GPD).sampling(data=data,iter=1000,warmup=200, chains=1) #we sample from the provided data ;
print(fit)

In [ ]:
beta = np.mean(list(fit.extract().values())[1].tolist())
alpha = np.mean(list(fit.extract().values())[0].tolist())
print("alpha = ", alpha, "\n beta = ", beta)
gamma = 1 / alpha 
def quantile_GPD(N, k, p, beta, gamma, u):
    return( u + beta*( ( N * p / k )**( -gamma ) - 1 ) ) # p = 0.05

quant_GPD = quantile_GPD(N, k, 0.05, beta, gamma, u)

In [ ]:
quant_GPD